# API RED ELECTRICA DE ESPAÑA

Vamos a obtener datos de la web de Red Eléctrica de España (https://www.ree.es/es) usando su API y siguiendo la información de la siguiente documentación (https://www.ree.es/es/apidatos).

Los datos a manejar serán los relativos a:

    1º Demanda de energia electrica a nivel nacional.
    2ª Generación de la misma a partir de distintas tecnologías tanto renovables como no renovables.
    3ª Las emisiones de Co2 de estas tecnologías.
    4ª Los precios de la energía y el mercado electrico.
    5º Los intercambios de energía con otros países colindantes.
    
    

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

import json
from pandas import Timestamp

import requests
from pprint import pprint

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
API_KEY = "keyYGla4KxFHZcthn" # Usuario actual: Adrián

BASE_ID = "appABmN3Mc59ZBaEh" # Base: Tabla API

TABLE_ID = "tblTbWDs3zjpNbDDL" # Tabla: datos_1

airtable_base_url = "https://api.airtable.com/v0"

headers = {"Authorization" : f"Bearer {API_KEY}",
               "Content-Type"  : "application/json"}

endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

# EXTRACCIÓN

## Automatización de la extracción

Automatizamos la extracción de la información con respecto a los diferentes mercados de la energía en España para cualquiera de las subcategorías que se deseen (salvo los precios en tiempo real, pues no aportan nueva información muy relevante y su extracción es mucho más específica, de manera que se complica innecesariamente esta parte del proyecto)

#### Nota: Asegúrese de no estar usando el widget 'precios-mercados-tiempo-real'

In [2]:
def extraccion_mercado(widgets):
    search_url = "https://apidatos.ree.es/es/datos/"
    category = "mercados"
    time_trunc="month"
    
    
    Data_gen=[]
    try:
        for x in range(2014,2024,2): # Solo podemos acceder a los datos en conjuntos de 24 meses en 24 meses
            start_date = f"{x}-05-31T00:00"
            end_date= f"{x+2}-05-30T23:59"

            query = f"start_date={start_date}&end_date={end_date}&time_trunc={time_trunc}"        
            endpoint = f"{search_url}{category}/{widgets}?{query}" # Accedemos a la categoría mercados, al widget seleccionado e insertamos el query
            print(f"Endpoint: {endpoint}")

            response = requests.get(url = endpoint)
            print(f"response: {response.status_code}")

            data = response.json()
            Data_gen.extend(data["included"][0]["attributes"]["values"])

        df = pd.DataFrame(Data_gen)
    
    except:
        print("Asegúrese de que no está usando 'precios-mercados-tiempo-real'")
    
    return df

In [3]:
df = extraccion_mercado("componentes-precio")

Endpoint: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2014-05-31T00:00&end_date=2016-05-30T23:59&time_trunc=month
response: 200
Endpoint: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2016-05-31T00:00&end_date=2018-05-30T23:59&time_trunc=month
response: 200
Endpoint: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2018-05-31T00:00&end_date=2020-05-30T23:59&time_trunc=month
response: 200
Endpoint: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2020-05-31T00:00&end_date=2022-05-30T23:59&time_trunc=month
response: 200
Endpoint: https://apidatos.ree.es/es/datos/mercados/componentes-precio?start_date=2022-05-31T00:00&end_date=2024-05-30T23:59&time_trunc=month
response: 200


In [4]:
df

,value,percentage,datetime
0,51.92,0.893786,2014-06-01T00:00:00.000+02:00
1,49.09,0.871936,2014-07-01T00:00:00.000+02:00
2,50.71,0.913858,2014-08-01T00:00:00.000+02:00
3,59.90,0.915902,2014-09-01T00:00:00.000+02:00
4,56.84,0.914414,2014-10-01T00:00:00.000+02:00
...,...,...,...
103,73.20,0.994160,2023-01-01T00:00:00.000+01:00
104,135.51,0.996837,2023-02-01T00:00:00.000+01:00
105,92.38,0.995581,2023-03-01T00:00:00.000+01:00
106,74.69,0.994143,2023-04-01T00:00:00.000+02:00


# Transformación

Procedemos a generalizar la limpieza del DataFrame, de manera que se elimina la columna 'percentage', pues nos es innecesaria; y limpiamos la columna datetime para que se renombre como Meses y queden reflejadas las fechas sin las horas ni las zonas horarias, de modo que sea más visualmente entendible y atractivo.

Finalmente, nombramos la columna 'value' como '€/Mwh', que es la unidad de referencia para los precios.

In [5]:
def limpieza_mercados(df):
    df.drop('percentage', axis = 1, inplace= True)
    
    df.rename(columns = {"value" : "€/Mwh", "datetime" : "Meses"}, inplace = True)
    
    df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x : x.split('T')[0])
    
    
    
    return df

In [6]:
limpieza_mercados(df)

,€/Mwh,Meses
0,51.92,2014-06-01
1,49.09,2014-07-01
2,50.71,2014-08-01
3,59.90,2014-09-01
4,56.84,2014-10-01
...,...,...
103,73.20,2023-01-01
104,135.51,2023-02-01
105,92.38,2023-03-01
106,74.69,2023-04-01


# Subida a AirTable

Una vez los datos quedan limpios, ya están listos para subirse a AirTable, por lo que automatizamos la subida de éstos, al igual que haremos a continuación con su descarga para futuros usos necesarios.

In [9]:
def upload_mercados(df):
    

    API_KEY = "keyYGla4KxFHZcthn" # Usuario actual: Adrián

    BASE_ID = "appABmN3Mc59ZBaEh" # Base: Tabla API

    TABLE_ID = "tblTbWDs3zjpNbDDL" # Tabla: datos_1

    airtable_base_url = "https://api.airtable.com/v0"

    headers = {"Authorization" : f"Bearer {API_KEY}",
               "Content-Type"  : "application/json"}

    endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

    datos_json = [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])]

    for i in range(0, df.shape[0], 10):

        data = {"records" : datos_json[i : i + 10], "typecast" : True}

        response = requests.post(url = endpoint, json = data, headers = headers)

In [10]:
upload_mercados(df)

# Descarga de datos de AirTable

In [40]:
def descarga(endpoint, headers):
    
    params = {}

    datos = []
    
    precio = []
    
    meses = []

    while params.get('offset') != None or len(datos) == 0:

        response = requests.get(url = endpoint, headers = headers, params = params)

        airtable_data = response.json()
        offset = airtable_data.get('offset')

        params['offset'] = offset

        datos.extend(airtable_data['records'])

    for i in range(len(datos)):
    
        precio.append(datos[i]['fields']['€/Mwh'])
        
        meses.append(datos[i]['fields']['Meses'])
    
    return precio, meses

In [41]:
descarga(endpoint, headers)

([135.51,
  50.84,
  26.74,
  55.41,
  50.77,
  83.94,
  24.85,
  38.5,
  66.2,
  59.9,
  41.63,
  67.95,
  143.18,
  92.8,
  61.86,
  41.75,
  202.01,
  73.56,
  45.37,
  36.54,
  44.17,
  156.53,
  43.6,
  202.47,
  52.63,
  35.39,
  51.78,
  21.7,
  51.92,
  53.04,
  42.06,
  117.35,
  53.54,
  43.93,
  44.62,
  46.39,
  54.38,
  58.86,
  142.69,
  62.94,
  187.85,
  65,
  36.75,
  49.14,
  287.23,
  35.2,
  74.67,
  43.47,
  48.04,
  49.35,
  42.74,
  205.86,
  53.79,
  48.68,
  154.77,
  54.93,
  55.52,
  63.6,
  43.55,
  60.54,
  47.6,
  47.4,
  42.59,
  41.07,
  92.38,
  106.45,
  45.91,
  62.32,
  62.63,
  29.86,
  62.98,
  169.73,
  44.2,
  197.46,
  28.8,
  194.38,
  60.53,
  52.68,
  28.28,
  28.65,
  37.46,
  56.71,
  71.78,
  17.81,
  49.43,
  48.93,
  30.99,
  74.69,
  50.94,
  46.59,
  55.77,
  57.59,
  39.29,
  73.2,
  47.74,
  42.91,
  245.71,
  101.36,
  44.24,
  50.71,
  60.14,
  49.55,
  66.1,
  51.96,
  128.83,
  49.09,
  56.84,
  57.4],
 ['2023-02-01',
  '2015-10-

# Visualización

## Contexto

En el mercado mayorista, los participantes realizan transacciones para asegurar el suministro de electricidad a corto plazo. Los productores de electricidad ofrecen su energía a un precio determinado, y los comercializadores y distribuidores realizan compras según sus necesidades y previsiones de demanda.

Posteriormente, estos precios se reflejan en la factura de electricidad de los consumidores finales, pero a través de un proceso regulado y controlado.

**El precio final que afrontan los consumidores NO es necesariamente el que se muestra en el mercado diario en un momento dado.**

## Serie temporal

Resulta interesante realizar un análisis longitudinal de la evolución de los precios en el Mercado Diario, de manera que observamos que se alcanzó el **mínimo** en el mes de **abril del 2020**, tras un descenso escalonado como subproducto de la **pandemia de la Covid-19**, mientras que el **máximo** se alcanzó en el mes de **marzo del 2022**, tras un volatil ascenso una vez termina dicha pandemia (y aumenta con ello la demanda) y, al mismo tiempo, inicia la **guerra de Ucrania** en febrero del 2022.

In [21]:
maximo = df[df['€/Mwh'] == df['€/Mwh'].max()]
minimo = df[df['€/Mwh'] == df['€/Mwh'].min()]


valores_extremos = pd.concat([maximo, minimo])


fig = px.line(df, x='Meses', y='€/Mwh')
fig.add_trace(px.scatter(valores_extremos, x='Meses', y='€/Mwh').data[0])

fig.show()

-------
-------
Corroboramos que nos hallamos ante una ***situación excepcional*** si comparamos los promedios entre los años *2014 y 2019*, siendo el **valor más repetido** aquél que fluctúa entre **50 y 55 euros** el megavatio hora, con un **máximo** de **66€/Mwh** y un **mínimo** de **30€/Mwh**. Si observamos entre los años *2020 a 2023*, veremos que **no hay un valor más repetido**, sino ***fuertes oscilaciones*** de los precios, llegando a un **máximo** de **214€/Mwh**

------
------

In [51]:
fig = px.histogram(df, x = 'Meses', y = '€/Mwh', histfunc = 'avg', nbins = 50)

fig.update_layout(
    title = 'Distribución de los precios medios de electricidad en España',
    xaxis_title = 'Meses',
    yaxis_title = 'Precio (€/MWh)')

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)', 
                  marker_line_width=1.5)

fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')

fig.update_traces(opacity=0.75)

fig.show()